In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import  numpy as np

tf.random.set_seed(22)
np.random.seed(22)

In [2]:
batchsz = 128

# the most frequest words
# 设定10000个常用单词， 标记（0~9999），其余单词都是生僻词，这些生僻词都标记为10000
total_words = 10000
# 设定句子长度80， 不足80补0，超过80截断
max_review_len = 80
# 一个单词使用100维表示，不是one-hot-encoding
embedding_len = 100
# 
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)
# x_train:[b, 80]
# x_test: [b, 80]
# 对数据padding,截取或补足
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# 如果不是batchsz整数倍，把最后一个batch删掉：drop_remainder
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batchsz, drop_remainder=True)
print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test shape: (25000, 80)


### SimpleRNNCell

In [3]:
class MyRNN(keras.Model):

    def __init__(self, units):
        super(MyRNN, self).__init__()

        # [b, 64]
        self.state0 = [tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units])]

        # transform text to embedding representation
        # 进行整句编码
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        # RNN: cell1 ,cell2, cell3
        # SimpleRNN
        self.rnn_cell0 = layers.SimpleRNNCell(units, dropout=0.5)
        self.rnn_cell1 = layers.SimpleRNNCell(units, dropout=0.5)


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)
    # 在调用fit和evaluate时， trainging参数会变化
    def call(self, inputs, training=None):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # [b, 80, 100] => [b, 64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1): # word: [b, 100]
            # h1 = x*wxh+h0*whh
            # out0: [b, 64]
            out0, state0 = self.rnn_cell0(word, state0, training)
            # out1: [b, 64]
            out1, state1 = self.rnn_cell1(out0, state1, training)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(out1)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

In [4]:
units = 64
epochs = 4
# 由于AMD DirectML的原因，不支持GPU
with tf.device("/cpu:0"):
    model = MyRNN(units)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
                    loss = tf.losses.BinaryCrossentropy(),
                    metrics=['accuracy'])
    model.fit(db_train, epochs=epochs, validation_data=db_test)

    model.evaluate(db_test)

Epoch 1/4
195/195 [==============================] - 18s 59ms/step - loss: 0.5832 - accuracy: 0.6727 - val_loss: 0.4354 - val_accuracy: 0.8026
Epoch 2/4
195/195 [==============================] - 10s 50ms/step - loss: 0.3783 - accuracy: 0.8389 - val_loss: 0.4100 - val_accuracy: 0.8249
Epoch 3/4
195/195 [==============================] - 10s 52ms/step - loss: 0.3045 - accuracy: 0.8776 - val_loss: 0.4060 - val_accuracy: 0.8315
Epoch 4/4
195/195 [==============================] - 3s 14ms/step - loss: 0.4722 - accuracy: 0.8224


### SimpleRNN layer



In [3]:
class MyRNN(keras.Model):

    def __init__(self, units):
        super(MyRNN, self).__init__()


        # transform text to embedding representation
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        self.rnn = keras.Sequential([
            layers.SimpleRNN(units, dropout=0.5, return_sequences=True, unroll=True),
            layers.SimpleRNN(units, dropout=0.5, unroll=True)
        ])


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training=None):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # x: [b, 80, 100] => [b, 64]
        x = self.rnn(x)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(x)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

In [4]:
units = 64
epochs = 4

with tf.device("/cpu:0"):
    model = MyRNN(units)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
                    loss = tf.losses.BinaryCrossentropy(),
                    metrics=['accuracy'])
    model.fit(db_train, epochs=epochs, validation_data=db_test)

    model.evaluate(db_test)

Epoch 1/4
195/195 [==============================] - 21s 69ms/step - loss: 0.5954 - accuracy: 0.6591 - val_loss: 0.4110 - val_accuracy: 0.8173
Epoch 2/4
195/195 [==============================] - 12s 59ms/step - loss: 0.3896 - accuracy: 0.8336 - val_loss: 0.3874 - val_accuracy: 0.8314
Epoch 3/4
195/195 [==============================] - 12s 60ms/step - loss: 0.3193 - accuracy: 0.8690 - val_loss: 0.3976 - val_accuracy: 0.8289
Epoch 4/4
195/195 [==============================] - 3s 17ms/step - loss: 0.4101 - accuracy: 0.8309
